# UAVSAR Data Products

UAVSAR has a variety of different type of images:

[Repeat Pass Interferometric](https://uavsar.jpl.nasa.gov/science/documents/rpi-format.html) images contain:
```{admonition} InSAR Data Types
:class: InSAR Data Types
- ANN file (.ann): a text annotation file with metadata
- AMP files (.amp1 and .amp2): amplitude products for flight 1 and flight 2
- COR files (.cor): coherence a measure of the noise level of the phase
- INT files (.int): wrapped phase difference between the two images
- UNW files (.unw): unwrapped phase difference between the two images
- INC files (.inc): incidence angle in radians
- HGT file  (.hgt): the DEM that was used in the InSAR processing
```

UAVSAR repeat pass interferometry uses two images of the same place but separated in time. Phase changes between the two aquistions are calculated,  creating a wrapped interferogram. These phase changes are due to either the wave traveling a longer distance (ground movement or refraction) or change wave speeds (atmospheric water vapor and snow).

- GRD files (.grd): products projected to the ground in geographic coordinates (latitude, longitude)
Finally all images can be in radar slant range or projected into WGS84. Images that have already been projected to ground range will have the extension .grd appended to their file type extension. 

For instance a image of unwrapped phase that has not been georefenced would end with .unw, while one that was georeferenced would end with .unw.grd. You will generally want to use .grd files for most analysis.


[Polarimetric PolSAR](https://uavsar.jpl.nasa.gov/science/documents/polsar-format.html) images contain:
- ANN file (.ann): a text annotation file with metadata
- Polsar file (_HHVV_.grd): all the rest of the files will be a pair of polarizations pushed together

Polsar files have a pair of polarizations (VV, VH, HV, HH) combined in their file name. These files are the phase difference between polarization XX and polarization YY. For instance HHHV is the phase difference between HH and HV polarizations. HVVV is the phase difference between HV and VV and so one. There are 6 of these pairs since order is irrelevant. These 6 images are combined to calculate various metrics that tell you about the types of scattering occurring.

## Import Libraries

In [ ]:
try:
    from uavsar_pytools import UavsarScene
except ModuleNotFoundError:
    print('Install uavsar_pytools with `pip install uavsar_pytools`')

import os
from os.path import join, basename
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import holoviews as hv
import rioxarray as rxa
from bokeh.plotting import show
import datashader as ds
from datashader.mpl_ext import dsshow
hv.extension('bokeh', logo=False)
import earthpy.plot as ep
import earthpy.spatial as es

## Interferometric Imagery

### Banner Summit

In this section we'll be plotting and comparing dirrerent types of SAR and InSAR data with optical imagery and a digital elevation model. For this example we'll be taking a subet of the Lowman flight (Boise, ID) line encompassing Banner Summit.


:::{figure-md} lowman
<img src="../../img/lowman_map.png" width="50%" style="background-color:white;">

Map of Lowman UAVSAR swath
:::

In [ ]:
# clone tifs to tmp directory
os.chdir('/tmp/')
!git clone https://github.com/SnowEx/uavsar-tutorial-data.git

In [ ]:
# list files downloaded
data_dir = '/tmp/uavsar-tutorial-data/'
os.chdir(data_dir)
! ls -l

# Load in Rasters
Here we'll load our rasters into the environemtns using ```rioxarray``` or ```rxa```, we will then convert to a ```np.array``` to be able to use ```matplotlib.pyplot``` or ```plt``` for plotting

## Optical Data

We will be using [Haromized Landsat Sentinel (HLS)](https://hls.gsfc.nasa.gov/) dataset from January 13th, 2021. This date was selected because it is mostly cloud free, which is uncommon in mountain environments during the winter.

In [ ]:
# define paths to the three RGB bands
red_path = 'lowman_red.tif'
green_path = 'lowman_green.tif'
blue_path = 'lowman_blue.tif'
stack_band_paths = (red_path,green_path,blue_path)
stack_band_paths

In [ ]:
# path to stacked raster
raster_out_path = "lowman_rgb.tif"

Using ```earthpy.spatial.stack``` or ```es.stack```, stack the three bands for plotting.

In [ ]:
# using earphy.spatial stack
stack = es.stack(stack_band_paths, out_path=raster_out_path)

In [ ]:
# load in stack using rioxarray
rgb = rxa.open_rasterio(raster_out_path)

# What do we see in this image? Any notable features?

In [ ]:
# plot rgb image
ep.plot_rgb(rgb.values,
            figsize=(15, 15),
            rgb = [0,1,2], # plot the red, green, and blue bands in that order
            title = "HLS Optical 2/18/2021", 
            stretch=True)
plt.show()

## InSAR and SAR Data

Here we'll be using five different data products related to InSAR and SAR: unwrapped phase (```unw```), coherence (```cor```), amplitude (```amp```), elevation (```dem```), and incidence angle (```inc```).

In [ ]:
# open raster and inspect meta data using xarray
unw_rast  = rxa.open_rasterio('lowman_unw.tif')
unw = unw_rast[0].values # np.array for plotting
    
# coherence
cor_rast  = rxa.open_rasterio('lowman_cor.tif')
cor = cor_rast[0].values

# amplitude
amp_rast  = rxa.open_rasterio('lowman_amb_db.tif')
amp = amp_rast[0].values # np.array for plotting

# dem
dem_rast  = rxa.open_rasterio('lowman_dem.tif')
dem = dem_rast[0].values

# incidence angle
inc_rast  = rxa.open_rasterio('lowman_inc_deg.tif')
inc = inc_rast[0].values # np.array for plotting

In [ ]:
# plot unwrapped phase

plt.rcParams.update({'font.size': 12}) # increase plot font size for larger plot
fig, ax = plt.subplots(figsize=(15, 15))

ax.set_title("UNW (radians)", fontsize= 20) #title and font size
img = ax.imshow(unw, interpolation = 'nearest', cmap = 'viridis', vmin = -3, vmax = 2)

# add legend
colorbar = fig.colorbar(img, ax=ax, fraction=0.03, pad=0.04) # add color bar
plt.show()

In [ ]:
# plot coherence

plt.rcParams.update({'font.size': 12}) # increase plot font size for larger plot
fig, ax = plt.subplots(figsize=(15, 15))

ax.set_title("Coherence", fontsize= 20) #title and font size
img = ax.imshow(cor, cmap = 'magma', vmin = 0, vmax = 1)

# add legend
colorbar = fig.colorbar(img, ax=ax, fraction=0.03, pad=0.04) # add color bar
plt.show()

In [ ]:
# plot amplitude

plt.rcParams.update({'font.size': 12}) # increase plot font size for larger plot
fig, ax = plt.subplots(figsize=(15, 15))

ax.set_title("Amplitude (dB)", fontsize= 20) #title and font size
img = ax.imshow(amp, cmap = 'Greys_r', vmin = -20, vmax = 0)

# add legend
colorbar = fig.colorbar(img, ax=ax, fraction=0.03, pad=0.04) # add color bar
plt.show()

In [ ]:
# plot dem

plt.rcParams.update({'font.size': 12}) # increase plot font size for larger plot
fig, ax = plt.subplots(figsize=(15, 15))

ax.set_title("Elevation (m)", fontsize= 20) #title and font size
img = ax.imshow(dem, cmap = 'terrain', vmin = 1800, vmax = 2800)

# add legend
colorbar = fig.colorbar(img, ax=ax, fraction=0.03, pad=0.04) # add color bar
plt.show()

In [ ]:
# plot incidence angle

plt.rcParams.update({'font.size': 12}) # increase plot font size for larger plot
fig, ax = plt.subplots(figsize=(15, 15))

ax.set_title("Incidence Angle (deg)", fontsize= 20) #title and font size
img = ax.imshow(inc, cmap = 'Spectral_r', vmin = 20, vmax = 90)

# add legend
colorbar = fig.colorbar(img, ax=ax, fraction=0.03, pad=0.04) # add color bar
plt.show()

## Comparison Plot

In [ ]:
# plot all InSAR products
fig = plt.figure(figsize=(30,19))

ax = fig.add_subplot(1,3,1)
cax=ax.imshow(unw, cmap='viridis', interpolation = 'nearest', vmin = -3, vmax = 2)
ax.set_title("UNW (radians)")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-3,0,2],orientation='horizontal', fraction=0.03, pad=0.04)
cbar.ax.set_xticklabels([-3,0,2])

ax = fig.add_subplot(1,3,2)
cax = ax.imshow(cor, cmap = 'magma', vmin = 0, vmax = 1)
ax.set_title("Coherence")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[0,.5,1], orientation='horizontal',fraction=0.03, pad=0.04)


ax = fig.add_subplot(1,3,3)
cax = ax.imshow(amp, cmap = 'Greys_r', vmin = -20, vmax = 0)
ax.set_title("Amplitude (dB)")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-20,-10,0], orientation='horizontal',fraction=0.03, pad=0.04)
cbar.ax.set_xticklabels([-20,-10,0])

ax = fig.add_subplot(2,3,1)
cax = ax.imshow(inc, cmap = 'Spectral_r', vmin = 20, vmax = 90)
ax.set_title("Incidence Angle (deg)")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[20,90], orientation='horizontal', pad=0.07)
cbar.ax.set_xticklabels([20,90])

ax = fig.add_subplot(2,3,2)
cax = ax.imshow(dem, cmap = 'terrain', vmin = 1800, vmax = 2800)
ax.set_title("Elevation (m)")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[1800,2800], orientation='horizontal', pad=0.07)
cbar.ax.set_xticklabels([1800,2800])

done = None

In [ ]:
ep.plot_rgb(rgb.values,
            figsize=(7, 7),
            rgb = [0,1,2], # plot the red, green, and blue bands in that order
            title = "HLS Optical 2/18/2021", 
            stretch=True)
plt.show()

## What are some notable similarities between images? Differences?

In the next section we'll go into more detail about the features that impact coherence, phase, and how they're related

## Sagehen Creek Example

In [ ]:
donner_dir = join(data_dir, 'sage')
# donner_dir = os.path.expanduser('~/Downloads/sage2')
imgs ={}
for fp in glob(join(donner_dir, '*.tif')):
    name = basename(fp).split('.')[0]
    imgs[name] = rxa.open_rasterio(fp, parse_coordinates=True, default_name = name)

## What topographic features seem to impact coherence?

Take a moment to chat with the people around you about this. Some features to get you thinking:

- lakes
- aspect (south vs north, east vs west)
- elevation
- trees
- roads
- others?

In [ ]:
tiles = hv.element.tiles.EsriUSATopo().opts()
cor = hv.Image(hv.Dataset(imgs['cor'], kdims=['x','y'])).opts(cmap = 'gray', colorbar=True, xaxis = None, yaxis = None, title = 'Coherence')
hgt = hv.Image(hv.Dataset(imgs['hgt'], kdims=['x','y'])).opts(cmap = 'terrain', colorbar=True, xaxis = None, yaxis = None, title= 'DEM', alpha = 0.4)
hgt_trans = hv.Image(hv.Dataset(imgs['hgt'][0,::100,::100], kdims=['x','y'])).opts(alpha = 0, xaxis = None, yaxis = None, title = 'Topo')
cor_tile = tiles  * cor
hgt_tile = tiles  * hgt
imagery = hv.element.tiles.EsriImagery()  * hgt_trans

hv.Layout([cor_tile, hgt_tile, imagery]).opts(width = 400, height = 900)

In [ ]:
import seaborn as sns
xna = imgs['hgt'].data.ravel()
yna = imgs['cor'].data.ravel()
x = xna[(~np.isnan(xna)) & (~np.isnan(yna))][::100]
y = yna[(~np.isnan(xna)) & (~np.isnan(yna))][::100]

df = pd.DataFrame(dict(x=x, y=y))
df['x_cat'] = pd.qcut(df.x, q= 6, precision = 0)
f, ax = plt.subplots(figsize = (12,8))
sns.violinplot(y = df.y[::100], x = df.x_cat[::100], scale = 'count')
plt.xlabel('Elevation Bands (m)')
plt.ylabel('Coherence')

## UNW vs. Coherence

In [ ]:
tiles = hv.element.tiles.EsriUSATopo().opts()
cor = hv.Image(hv.Dataset(imgs['cor'], kdims=['x','y'])).opts(cmap = 'gray', colorbar=True, xaxis = None, yaxis = None, title = 'Coherence')
unw = hv.Image(hv.Dataset(imgs['unw'], kdims=['x','y'])).opts(cmap = 'magma', colorbar=True, xaxis = None, yaxis = None, title= 'Unwrapped Phase', clim = (0, 2*np.pi))
cor_tile = tiles  * cor
unw_tile = tiles  * unw

hv.Layout([cor_tile, unw_tile]).opts(width = 400, height = 900)